<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>MetPy Foundations</h1>
    <h3>AMS 2022 Short Course: MetPy for Quantitative Analysis of Meteorological Data</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

### Tasks
1. <a href="#units">Units and MetPy</a>
1. <a href="#constants">MetPy Constants</a>
1. <a href="#calculations">MetPy Calculations</a>

<a name="background"></a>
## Background
MetPy is a modern meteorological open-source toolkit for Python. It is a maintained project of [Unidata](http://www.unidata.ucar.edu) to serve the academic meteorological community. MetPy consists of three major areas of functionality:


### Plotting
As meteorologists, we have many field specific plots that we make. Some of these, such as the Skew-T Log-p require non-standard axes and are difficult to plot in most plotting software. In MetPy we've baked in a lot of this specialized functionality to help you get your plots made and get back to doing science. We will go over making different kinds of plots during the workshop.<br>
<center><img width="700" src="https://unidata.github.io/MetPy/latest/_images/sphx_glr_Station_Plot_001.png"/><br><i>Example of MetPy plotting tools</i></center>

### Calculations
Meteorology also has a common set of calculations that everyone ends up programming themselves. This is error-prone and a huge duplication of work. MetPy contains a set of well-tested calculations that is continually growing in an effort to be at feature parity with other legacy packages such as GEMPAK.
<center><a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html" target="blank">MetPy Calculation Reference</a></center>

### File I/O
Finally, there are a number of odd file formats in the meteorological community. MetPy has incorporated a set of readers to help you deal with file formats that you may encounter during your research, including working with many xarray functions for data organization. 
<center><a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.io.html" target="blank">MetPy I/O Reference</a> | <a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.xarray.html" target="blank">MetPy xarray Reference</a></center>


<a name="units"></a>
## Units and MetPy

Early in our scientific careers we all learn about the importance of paying attention to units in our calculations. Unit conversions can still get the best of us and have caused more than one major technical disaster, including the crash and complete loss of the $327 million [Mars Climate Orbiter](https://en.wikipedia.org/wiki/Mars_Climate_Orbiter). 

MetPy uses the [pint](https://pint.readthedocs.io/en/latest/) library and a custom unit registry to help prevent unit mistakes in calculations. That means that every quantity you pass to MetPy should have units attached, just like if you were doing the calculation on paper. Attaching units is easy, simply multiply (`*`) the magnitude by the units in the format `units.___`.

In [40]:
# Import the MetPy unit registry
from metpy.units import units

In [17]:
length = 10.4 * units.inches
width = 20 * units.meters

print(length, width)

10.4 inch 20 meter


You can also use tab completion to see what units are available in the `units` registry.

Let's now attempt a rectangular area calculation with the above measurements. Multiplying length and width, we'll get an area in return with units attached.

In [18]:
area = length * width

print(area)

208.0 inch * meter


That's great, now we have an area, but it is not in a very useful unit...

<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate the area of a rectangle

Calculate the area of the above rectangle in square meters ($m^2$) using any method you can think of.
</div>

In [ ]:
# YOUR CODE GOES HERE

Assuming you did not look ahead (or know this MetPy function previously), you probably had to look up the conversion factor between inches and meters and used a method similar to this:

`length = 10.4 * 0.0254 * units.meters`

`area = length * width`

MetPy can save you the headache of looking up conversions and maintaining high precision with the `.to()` method. 

You have the option of converting the individual measurements or the final area calculation. While you won't see m$^2$ in the units list, we can parse complex/compound units as strings:

In [19]:
area.to('m^2')

5.2832 <Unit('meter ** 2')>

<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate distance

Calculate the distance a parcel traveling 50 knots due east would traverse in 1 day    
    <ul>
         <li>Create a variable named <code>speed</code> with a value of 50 knots.</li>
         <li>Create a variable named <code>time</code> with a value of 1 day.</li>
         <li>Calculate the distance traveled in kilometers.</li>
    </ul>
</div>

In [ ]:
# YOUR CODE GOES HERE

<div class="alert alert-info">
    <b>SOLUTION</b>
</div>

In [21]:
speed = 50 * units.knot
time = 1 * units.day

distance = speed * time 
distance.to('km')

# roughly the distance between LA and Little Rock

2222.4000000000005 <Unit('kilometer')>

When performing a series of calculations with units, it is best practice to group the scalar magnitude with the units in parentheses. This is especially important for division and rate calculations to ensure the correct unit output. Compare the following examples. Which one outputs the correct units for velocity?

In [38]:
v1 = 10 * units.m / 20 * units.s
print("v1: " + (str(v1))) 

v2 = 10 * units.m / (20 * units.s)
print("v2: " + (str(v2)))

v1: 0.5 meter * second
v2: 0.5 meter / second


### Temperature
In meteorology, we frequently use three different measurements systems for temperature. We often get temperature in Kelvin from model output, but may want to report temperature for communication purposes in Celsius or Fahrenheit. To convert from one unit of temperature to another, we apply a conversion equation such as: $$F = (K − 273.15) * (9/5) + 32$$


These conversion are straightforward for simple one to one calculation. Where we run into trouble is when we refer to changes in temperature from one unit system to another. Temperature is a non-multiplicative unit - they are in a system with a reference point. That means that not only is there a scaling factor, but also an offset. This makes the math and unit book-keeping a little more complex. 

Imagine running a numerical model that tests the effect of surface temperature on cloud cover. Let's say we want to increase the baseline surface temperature of 290 K by 8 degrees Celsius. There may be many ways you can think of to complete this operation, so let's test a few methods:




Imagine adding 10 degrees Celsius to 100 degrees Celsius. Is the answer 110 degrees Celsius or 383.15 degrees Celsius (283.15 K + 373.15 K)? That's why there are delta degrees units in the unit registry for offset units. For more examples and explanation you can watch [MetPy Monday #13](https://www.youtube.com/watch?v=iveJCqxe3Z4).



In [9]:
# Starting simple, try adding 8 degrees C to 290 K
290 * units.kelvin + 8 * units.degC

OffsetUnitCalculusError: Ambiguous operation with offset unit (kelvin, degree_Celsius). See https://pint.readthedocs.io/en/latest/nonmult.html for guidance.

Notice that this fails with error `"Ambiguous operation with offset unit"` because we cannot add two units with offset reference points. 

Instead, we must look again at the problem we are trying to solve: Increase 290 K by 8 degrees Celsius. In this case, the 8 degrees Celsius is not a single temperature measurement, it is a representation of temperature change. On the Kelvin scale, we increase the starting temperature by an equivalent of 8 degrees Celsius, i.e. $\Delta$ 8 degrees Celsius.

MetPy (and pint) have a special unit to complete these kinds of calculations, `delta_degC`. Let's try our calculation again and find our resulting surface temperature:

In [10]:
290 * units.kelvin + 8 * units.delta_degC

298 <Unit('kelvin')>

<div class="alert alert-warning">
    <b>NOTE</b>
    
Absolute temperature scales like Kelvin and Rankine do not have an offset and therefore can be used in addition/subtraction without the need for a delta verion of the unit. For example, 
    
<code>273 * units.kelvin + 10 * units.kelvin
  283 kelvin</code>
    
<code>273 * units.kelvin - 10 * units.kelvin
  263 kelvin</code>
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>: Temperature change

A parcel at 60 degrees Fahrenheit is lifted dry adiabatically from the surface to a level of 1500 meters above ground level. 
    
Assuming a dry adiabatic lapse rate of -10 degrees C per 1000 meters, what is the final temperature of the parcel after lifting?
    
<b>Hint:</b> Remember to group your units and scalar magnitude with parentheses.
    
<b>Bonus:</b> Assuming a moist adiabatic lapse rate of -6 degrees C per 1000 meters, what is the temperature of the parcel if it continues lifting moist adiabatically an additional 2000 meters? (Final elevation of 3500 meters)

</div>

In [ ]:
# YOUR CODE GOES HERE

<div class="alert alert-info">
    <b>SOLUTION</b>
</div>

In [45]:
# define lapse rate
dalr = -10 * units.delta_degC / (1000 * units.meters)

# define starting temperature
parcel_t = 60 * units.degF

# lifting
parcel_t = parcel_t + dalr * (1500 * units.meters)
print("Parcel temp after dry adiabatic lift for 1500 m: " + str(parcel_t) + ", " + str(parcel_t.to('degC')))

# Bonus:
malr = -6 * units.delta_degC / (1000 * units.meters)
parcel_t = parcel_t + malr * (2000 * units.meters)
print("Parcel temp after moist adiabatic lift for additional 2000 m: " + str(parcel_t) + ", " + str(parcel_t.to('degC')))

Parcel temp after dry adiabatic lift for 1500 m: 33.0 degree_Fahrenheit, 0.5555555555555998 degree_Celsius
Parcel temp after moist adiabatic lift for additional 2000 m: 11.400000000000002 degree_Fahrenheit, -11.4444444444444 degree_Celsius


<a href="#top">Top</a>
<hr style="height:2px;">

<a name="constants"></a>
## MetPy Constants
Another common place that problems creep into scientific code is the value of constants. Can you reproduce someone else's computations from their paper? Probably not unless you know the value of all of their constants. Was the radius of the earth 6000 km, 6300km, 6371 km, or was it actually latitude dependent?

MetPy has a set of constants that can be easily accessed and make your calculations reproducible. You can view a [full table](https://unidata.github.io/MetPy/latest/api/generated/metpy.constants.html#module-metpy.constants) in the docs, look at the module docstring with `metpy.constants?` or checkout what's available with tab completion.

In [41]:
import metpy.constants as mpconst
import numpy as 

In [12]:
mpconst.earth_avg_radius

6371008.7714 <Unit('meter')>

In [13]:
mpconst.dry_air_molecular_weight

0.02896546 <Unit('kilogram / mole')>

You may also notice in the table that most constants have a short name as well that can be used:

In [14]:
mpconst.Re

6371008.7714 <Unit('meter')>

In [15]:
mpconst.Md

0.02896546 <Unit('kilogram / mole')>

<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate Coriolis force

Calculate the Coriolis force (per unit mass) on a parcel moving 5 kts due east at your latitude (in units of degrees) using:<br>$$Coriolis Force = 2 \Omega V sin(\phi)$$
recalling that $V$ should be in units of m/s for this calculation. 
</div>

In [ ]:
# YOUR CODE GOES HERE

<div class="alert alert-info">
    <b>SOLUTION</b>
</div>

In [44]:
# Coriolis Force
v = 5 * units.knots
phi = 40 * units.degrees
cof = 2 * mpconst.omega * v.to('m/s') * np.sin(phi)
cof

0.00024113457576661478 <Unit('meter * radian / second ** 2')>

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="calculations"></a>
## MetPy Calculations
MetPy also encompasses a set of calculations that are common in meteorology (with the goal of have all of the functionality of legacy software like GEMPAK and more). The [calculations documentation](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html) has a complete list of the calculations in MetPy.

We'll scratch the surface and show off a few simple calculations here, but will be using many during the workshop.

In [24]:
import metpy.calc as mpcalc
import numpy as np

In [27]:
# Make some fake data for us to work with
np.random.seed(19990503)  # So we all have the same data
u = np.random.randint(0, 15, 10) * units('m/s')
v = np.random.randint(0, 15, 10) * units('m/s')

print(u)
print(v)

[14.0 2.0 12.0 5.0 3.0 5.0 14.0 8.0 9.0 10.0] meter / second
[6.0 10.0 7.0 11.0 10.0 13.0 2.0 3.0 5.0 0.0] meter / second


Let's use the `wind_direction` function from MetPy to calculate wind direction from these values. Remember you can look at the docstring or the website for help.

In [28]:
direction = mpcalc.wind_direction(u, v)
print(direction)

[246.80140948635182 191.30993247402023 239.74356283647072 204.44395478041653 196.69924423399362 201.03751102542182 261.86989764584405 249.44395478041653 240.94539590092285 270.0] degree


<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate wind speed

Calculate the wind speed using the <a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.wind_speed.html" target="blank">wind_speed function</a> in both m/s and mph.
</div>

In [ ]:
# YOUR CODE GOES HERE

<div class="alert alert-info">
    <b>SOLUTION</b>
</div>

In [35]:
# Wind speed
speedms = mpcalc.wind_speed(u,v).to('m/s')
print(speedms)

speedmph = mpcalc.wind_speed(u,v).to('mph')
print(speedmph)

[15.231546211727817 10.198039027185569 13.892443989449804 12.083045973594572 10.44030650891055 13.92838827718412 14.142135623730951 8.54400374531753 10.295630140987 10.0] meter / second
[34.0719985051177 22.81236360769857 31.076512145333307 27.029004056895516 23.354300529953807 31.156917227058244 31.63505642387918 19.11239205734952 23.030668711943 22.36936292054402] mile_per_hour


<a href="#top">Top</a>
<hr style="height:2px;">